# QRT ENS Data Challenge 2023 - Benchmark (english)

This notebook details the benchmark construction for the challenge - it may also help participants to start the competition.

## Libraries

In [79]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

## Loading data

- `X_train` and `X_test` both have $35$ columns that represent the same explanatory variables but over different time periods. 

- `X_train` and `Y_train` share the same column `ID` - each row corresponds to a unique ID associated wwith a day and a country. 

- The target of this challenge `TARGET` in `Y_train` corresponds to the price change for daily futures contracts of 24H electricity baseload. 

- **You will notice some columns have missing values**.


In [80]:
# After downloading the X_train/X_test/Y_train .csv files in your working directory:

X_train = pd.read_csv('../data/X_train.csv')
Y_train = pd.read_csv('../data/Y_train.csv')
X_test = pd.read_csv('../data/X_test.csv')

In [81]:
X_train.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378


In [82]:
Y_train.head()

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733


## Model and train score

The benchark for this challenge consists in a simple linear regression, after a light cleaning of the data: The missing (NaN) values are simply filled with 0's and the `COUNTRY` column is dropped - namely we used the same model for France and Germany. 

In [83]:
from sklearn.model_selection import train_test_split
lr = LinearRegression()

X_train_clean = X_train.drop(['COUNTRY'], axis=1).fillna(0)
Y_train_clean = Y_train['TARGET']


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_train_clean, Y_train_clean, test_size=0.33,random_state=40)

lr.fit(X_train, y_train)

output_train = lr.predict(X_train)
output_test= lr.predict(X_test)
def metric_train(output,y):

    return  spearmanr(output, y).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train,y_train) ))
print('Spearman correlation for the test set: {:.1f}%'.format(100 * metric_train(output_test, y_test)))

Spearman correlation for the train set: 29.7%
Spearman correlation for the test set: 17.7%


The Spearman correlation obtained with our model on the train data set is about 27.9%.

NB: Electricity price variations can be quite volatile and this is why we have chosen the Spearman rank correlation as a robust metric for the challenge, instead of the more standard Pearson correlation.

## Generate the benchmark output

Next, we process the test set the same way as we did on the train set and predict using our linear model, while saving the predictions to a csv file satisfying the challenge output contraints.


In [85]:
X_test.head()

,ID,DAY_ID,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,FR_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
81,603,641,-0.240316,-1.010226,-0.791374,0.791374,-1.384389,1.284447,1.384389,-1.284447,...,-0.906204,0.056528,-0.186613,0.595079,0.046315,-0.322930,-0.530443,-0.872621,0.683049,1.011981
784,1375,315,1.081157,1.492652,1.480474,-1.480474,-0.305735,-1.649489,0.305735,1.649489,...,1.434461,0.097601,-0.540710,-0.867803,-0.795372,0.041925,-0.776128,0.839717,-0.163634,0.041263
376,1099,141,0.338351,-0.512616,-1.332143,1.332143,0.000000,1.049176,0.000000,-1.049176,...,-0.353623,0.622641,-0.633195,-1.084618,-0.726670,-0.460941,-0.216763,-0.287806,-5.706442,-1.553390
70,549,587,0.005455,-0.826909,-1.127382,1.127382,-1.627090,1.057651,1.627090,-1.057651,...,-0.745098,-1.705716,-0.514725,0.533894,0.500430,0.023844,-0.083203,2.264979,1.357395,0.820938
482,44,901,0.149344,-0.551669,-1.660673,1.660673,-0.823389,1.332525,0.823389,-1.332525,...,-0.717190,-0.892206,-0.603828,-0.817149,-0.536019,0.030465,-0.951325,-0.155025,0.201747,0.765443


In [86]:
X_test_clean = X_test.drop(['COUNTRY'], axis=1).fillna(0)

Y_test_submission = X_test[['ID']].copy()
Y_test_submission['TARGET'] = lr.predict(X_test_clean)

Y_test_submission.to_csv('benchmark_qrt.csv', index=False)


KeyError: "['COUNTRY'] not found in axis"

After submitting this benchmark_qrt.csv file at https://challengedata.ens.fr, we obtain a public score of around 15.9 % - this is the score to beat!


## Tips and ideas for improvements 

- Thinking about the fundamental price drivers of electricity in each country may be useful. 

- The dataset is small, so you will need to be careful not to overfit your models. It will be useful to use good crossvalidation practices. 
